<a href="https://colab.research.google.com/github/ukab23/-Simple-Linear-Regression/blob/master/bnp_paribhas_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdqa

     |████████████████████████████████| 51kB 2.1MB/s 
     |████████████████████████████████| 133kB 8.2MB/s 
     |████████████████████████████████| 655kB 44.0MB/s 
  Created wheel for cdqa: filename=cdqa-1.1.0-cp36-none-any.whl size=46984 sha256=392d0e908d6b516120a7c205bd5d4618d034c32a189b778ac5eff61fe4d5b8d6
  Stored in directory: /root/.cache/pip/wheels/fe/b9/b8/f10ac30dc8559c5d1b3016301d4323c20f9dcc07ac3be25bd0
  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29223 sha256=c0b37674deb5fc0ba506e2d6fbe9d5ccf1022eaeaa0a3f05578b5ab379cebf7c
  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=a5610abe3add8a1c2d3d68dd8ee2ee7f152dfc3403bd49be7d9b1a11eac21b6c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for regex: filename=regex-2019.6.8-cp36-cp36m-linux_x86_64.wh

In [1]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline.cdqa_sklearn import QAPipeline

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')



bert_qa_vCPU-sklearn.joblib already downloaded


In [3]:
df = pd.read_csv('./data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

,date,title,category,link,abstract,paragraphs
0,13.05.2019,The banking jobs : Assistant Vice President – ...,Careers,https://group.bnpparibas/en/news/banking-jobs-...,Within the Group’s Corporate and Institutional...,[I manage a team in charge of designing and im...
1,13.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[With François Hollande, Chairman of French fo..."
2,13.05.2019,"""The bank with an IT budget of more than EUR6 ...",Group,https://group.bnpparibas/en/news/the-bank-budg...,"Interview with Jean-Laurent Bonnafé, Director ...","[We did the groundwork between 2012 and 2016, ..."
3,10.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[As part of the ‘United Tech of Europe’ theme,..."
4,10.05.2019,When Artificial Intelligence participates in r...,Careers,https://group.bnpparibas/en/news/artificial-in...,As the competition to attract talent intensifi...,[Online recruitment is already the norm. Accor...


In [4]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa_vCPU-sklearn.joblib')
cdqa_pipeline.fit_retriever(X=df)

100%|██████████| 231508/231508 [00:00<00:00, 1155259.12B/s]


QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir='models', predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1))

In [7]:
query = 'Since when does the Excellence Program of BNP Paribas exist?'
prediction = cdqa_pipeline.predict(X=query)

3it [00:00, 1373.23it/s]


In [9]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: Since when does the Excellence Program of BNP Paribas exist?
answer: January 2016
title: BNP Paribas’ commitment to universities and schools
paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.

